In [ ]:
! pip install pyowm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
METER_DIR = "/content/drive/My Drive/meters/" #'./data/meter_data/'
WEATHER_DIR = "/content/drive/My Drive/weather/" #'./data/weather_data/'
MAIN_METERS = "/content/drive/My Drive/main_meter_reads.csv" # until we get raw 20208-2019 data

import os
import numpy as np
import pandas as pd
import json
import time
import pyowm
from datetime import datetime, timedelta
from urllib.request import urlopen

In [ ]:
# Load main meter data files - all the same format - Andrea can just add additional files / replace files
############################################

# NOTE: This crawls through the meter directory to get all files
list_meter = []
meters = os.listdir(METER_DIR)

# loop through meters directory
for meter in meters: # to avoid ipynb checkpoints
  if meter != '.ipynb_checkpoints':
    df = pd.read_csv(METER_DIR+meter, index_col=None, header=0)
    list_meter.append(df)

df_meter = pd.concat(list_meter, axis=0, ignore_index=True)

In [ ]:
############################################
#
# Group sum the data by 30 minutes.
#
############################################

df_meter["READ_DTM"] = pd.to_datetime(df_meter["READ_DTM"], format='%m/%d/%Y %H:%M')
df_meter = df_meter.groupby(['METER_ID',pd.Grouper(key='READ_DTM', offset='0min', closed='right', freq='30min')]).sum()[["READ_VALUE"]].reset_index()
df_meter.tail(5)

,METER_ID,READ_DTM,READ_VALUE
377584,4440177,2020-11-01 21:30:00,2681.461548
377585,4440177,2020-11-01 22:00:00,2591.665161
377586,4440177,2020-11-01 22:30:00,2568.092651
377587,4440177,2020-11-01 23:00:00,2551.837403
377588,4440177,2020-11-01 23:30:00,2523.771850


In [ ]:
# Load weather data files - reread the updates files
############################################

# Add the pulled weather data to the dataframe
list_weather = []
weather = os.listdir(WEATHER_DIR)

# loop through weather directory
for w in weather: # to skip ipynb file that is first in the directory
    if w != '.ipynb_checkpoints': # we can remove this once this is a python file
      # this file was cleaned up, so we don't need to skiprows
      if w == 'BMG_hist.txt':
        df = pd.read_csv(WEATHER_DIR+w, index_col=None, header=0)
        list_weather.append(df)
      # we need to skip the first 5 rows in the text file
      else:
        df = pd.read_csv(WEATHER_DIR+w, index_col=None, header=0,skiprows=5)
        list_weather.append(df)

hist_weather = pd.concat(list_weather, axis=0, ignore_index=True)
hist_weather.drop("Unnamed: 0",inplace=True,axis=1)
hist_weather.tail(3)

,station,valid,tmpf,dwpf,relh
43994,BMG,2020-11-16 21:53,54.00,25.00,32.26
43995,BMG,2020-11-16 22:53,52.00,25.00,34.72
43996,BMG,2020-11-16 23:53,50.00,25.00,37.39


In [ ]:
############################################

# Historical weather data is already uploaded

############################################
#
# Get the last weather data since the last date
#
############################################
hist_weather["valid_dt"] = pd.to_datetime(hist_weather["valid"], format='%Y/%m/%d %H:%M')
last_date = hist_weather["valid_dt"].max()
last_date = datetime.utcfromtimestamp(datetime.timestamp(last_date))
yesterday = datetime.utcfromtimestamp(datetime.timestamp(datetime.now()+timedelta(days = 1)))

In [ ]:
# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
NETWORK = 'IN_ASOS'
STATION = 'BMG'


def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    print(uri)
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""

In [ ]:
def get_weather_history(startts, endts):
    """
    This function gets the defined weather station's tempurature, dewpoint
    and relative humidity for the date range specified in UTC and then 
    writes to a file in the format STATION_FromDate_ToDate.txt.
    Inputs:
        startts:    Starting Timestamp in UTC to request data for
        endts:      Ending Timestamp in UTC
    """

    service = SERVICE + "data=tmpf&data=relh&data=dwpf&tz=Etc/UTC&format=comma&latlon=no&"

    service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
    service += endts.strftime("year2=%Y&month2=%m&day2=%d&")
    
    station = STATION
    uri = "%s&station=%s" % (service, station)
    print("Downloading: %s" % (station,))
    data = download_data(uri)
    outfn = WEATHER_DIR + "%s_%s_%s.txt" % (
        station,
        startts.strftime("%Y%m%d%H%M"),
        endts.strftime("%Y%m%d%H%M"),
    )
    out = open(outfn, "w")
    out.write(data)
    out.close()

In [ ]:
# retrieve forecast data
recent_data = get_weather_history(startts=last_date, endts=yesterday)

In [ ]:
# Load weather data files - reread the updates files
############################################

# Add the pulled weather data to the dataframe
list_weather = []
weather = os.listdir(WEATHER_DIR)

# loop through weather directory
for w in weather: # to skip ipynb file that is first in the directory
    if w != '.ipynb_checkpoints': # we can remove this once this is a python file
      # this file was cleaned up, so we don't need to skiprows
      if w == 'BMG_hist.txt':
        df = pd.read_csv(WEATHER_DIR+w, index_col=None, header=0)
        list_weather.append(df)
      # we need to skip the first 5 rows in the text file
      else:
        df = pd.read_csv(WEATHER_DIR+w, index_col=None, header=0,skiprows=5)
        list_weather.append(df)

hist_weather = pd.concat(list_weather, axis=0, ignore_index=True)
hist_weather[["tmpf","dwpf","relh"]] = hist_weather[["tmpf","dwpf","relh"]].replace('M',np.nan)
hist_weather['tmpf'] = pd.to_numeric(hist_weather['tmpf'])
hist_weather['dwpf'] = pd.to_numeric(hist_weather['dwpf'])
hist_weather['relh'] = pd.to_numeric(hist_weather['relh'])
hist_weather['DTTM'] = pd.to_datetime(hist_weather['valid'])
hist_weather.index = hist_weather['DTTM']
hist_weather.drop(columns=['Unnamed: 0','station', 'valid', 'DTTM'], inplace=True)
hist_weather = hist_weather.resample('0.5H').mean().interpolate(method='time')
hist_weather = hist_weather.reset_index()

In [ ]:
############################################
#
# Calculate a rate column (double the usage)
#
#  NOTE:  This needs work - I grabbed it from the Peak Demand Fcst (AH).ipynb notebook which did some
#   basic manipulation to get the dataset working, so df refers to main_meter_reads and df_2020 refers
#   to the 2020_campus master meters.csv and weather_2020 is the IU weather 2020.csv file that Eric made.
#
#  OK - this is out of order, because this is working with the old main_meter_reads, which has the
#  KWH_RATE in the READ_VALUE column and already has weather data, while the raw meter files from
#  Andrea contain the actual READ_VALUE which we (after summing to 30 minutes) double to produce a
#  KWH_RATE column.  This is messed up a bit because we don't have the raw files from 2018/2019, just
#  the main_meter_reads.csv.  Also, that file messed with the dates, because it is missing some data
#  for Daylight Savings, yet it appears that the raw data Andrea provides is in EST (not EDT - there 
#  are 24 hours in every day).  The weather data comes in UTC, so you have to adjust 5 hours to get to 
#  EST.
#
############################################

# Join weather with meter data
df_meter["date_string"] = df_meter["READ_DTM"].astype(str)
hist_weather["date_string"] = hist_weather["DTTM"].astype(str)
df_agg = pd.merge(df_meter, hist_weather, how ='left', on ='date_string')

# select fields of interest
df_agg = df_agg[["METER_ID","READ_VALUE","date_string","READ_DTM","tmpf","dwpf","relh"]]

# Add a column to hold the kWh rate (from which Peak is determined) by taking the 30
# minute usage total and doubling it (what it would be for 60 minutes if the usage was the same)
df_agg['KWH_RATE'] = df_agg['READ_VALUE'] * 2

# Fix the outliers on Jan 1, 2019, which appears to be a doubling of the data
df_agg.loc[df_agg['date_string'] == '2019-01-01 00:00:00', 'READ_VALUE'] = df_agg.loc[df_agg['date_string'] == '2019-01-01 00:00:00', 'READ_VALUE'] / 2
df_agg = df_agg[["METER_ID","READ_VALUE","date_string","READ_DTM","tmpf","dwpf","relh"]]

# Add a column to hold the kWh rate (from which Peak is determined) by taking the old READ_VALUE,
# because that's what it was.  Then make the READ_VALUE the usage for the 30 minutes by 
# cutting it in half.
df_agg['KWH_RATE'] = df_agg['READ_VALUE']
df_agg['READ_VALUE'] = df_agg['KWH_RATE'] / 2

In [ ]:
#######################################################
# As of now we are dropping null values, 
#six missing, may want to improve upon it in the future
#######################################################
df_agg.dropna(axis=0,inplace=True)

In [ ]:
############################################
#
# Get the forecast weather data for the next 5 days from the end of the current time.  Interpolate to every 30 minutes.
#
#  NOTE:  This comes from IU_Peak_Test_Dataset.py.  I have not tested this or modified it, so it will likely need to be tweaked.
#
############################################

owm = pyowm.OWM('02d982dc96589daaeebb1a7fd826930f') # You MUST provide a valid API key
###############################################################################

# Get forecast
fc = owm.three_hours_forecast('Bloomington,IN,US')
f = fc.get_forecast()
###############################################################################

# Set right timezone -> I replaced datetime.now() with the latest date from the weather file
hour_difference = round((f.get_reception_time('date').replace(tzinfo=None) - datetime.now().replace(tzinfo=None)).seconds/3600)
###############################################################################

In [ ]:
# Start Creating Dataset

cols = ['datetime','date','month','day','hour','weekday','temperature','precipitation','wind_speed','humidity','conditions']

df = pd.DataFrame(columns=cols, index=range(0,40))

n = 0
for weather in f: ## fix hour
    data = [weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference), (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference)).date(), 
            (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference)).month, (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference)).day, 
            (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference)).hour, (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference)).weekday(), 
            weather.get_temperature('fahrenheit')['temp'], (0 if len(weather.get_rain()) == 0 else weather.get_rain()['3h']) + (0 if len(weather.get_snow()) == 0 else weather.get_snow()['3h']),
            weather.get_wind()['speed'], weather.get_humidity(), weather.get_status()]  
    df.iloc[n,:] = data
    n += 1
      # does it always do GMT 12/3/6/9/12/3/6/9 or does it change based on hour pull?
      # 

#############33333 add date to be dropped later, and weekday and remember to add offsets to hour, day, month

In [ ]:
# Turn 3hr forecast to hourly

df_lower = pd.DataFrame(columns=cols, index=range(0,40))

n = 0
for weather in f:
    data = [weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference+1), (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference+1)).date(), 
            (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference+1)).month, (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference+1)).day, 
            (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference+1)).hour, (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference+1)).weekday(), 
            weather.get_temperature('fahrenheit')['temp'], (0 if len(weather.get_rain()) == 0 else weather.get_rain()['3h']) + (0 if len(weather.get_snow()) == 0 else weather.get_snow()['3h']),
            weather.get_wind()['speed'], weather.get_humidity(), weather.get_status()]   
    df_lower.iloc[n,:] = data
    n += 1
    
df_higher = pd.DataFrame(columns=cols, index=range(0,40))

n = 0
for weather in f:
    data = [weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference-1), (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference-1)).date(), 
            (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference-1)).month, (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference-1)).day, 
            (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference-1)).hour, (weather.get_reference_time('date').replace(tzinfo=None) - timedelta(hours=hour_difference-1)).weekday(), 
            weather.get_temperature('fahrenheit')['temp'], (0 if len(weather.get_rain()) == 0 else weather.get_rain()['3h']) + (0 if len(weather.get_snow()) == 0 else weather.get_snow()['3h']),
            weather.get_wind()['speed'], weather.get_humidity(), weather.get_status()]   
    df_higher.iloc[n,:] = data
    n += 1

########################################################